# Flash Attention

视频一：https://www.bilibili.com/video/BV1UT421k7rA/?spm_id_from=333.337.search-card.all.click&vd_source=071b23b9c7175dbaf674c65294124341

主要靠的是改进IO从而提高速度  
分块与重计算

<div align="center">
    <img src="flashattention/硬件示意图.png" alt="图片描述" width="400"/>
</div>

<div align="center">
    <img src="flashattention/速度对比图.png" alt="图片描述" width="400"/>
</div>


原始Attention实现：

<div>
    <img src="flashattention/原始Attention实现.png" alt="图片描述" width="400"/>
</div>

时间消耗分为两类：
1. 计算时间：大的矩阵乘法、卷积等
2. IO时间（Memory）：（1）按位运算：ReLU、Dropout；（2）归约运算：sum，softmax  

主要优化IO时间，通过分块与重计算来减少IO次数

重点关注视频中softmax如何进行分块


## flash-attention2改进：

<div>
    <img src="flashattention/flash-attention2改进.png" alt="图片描述" width="400"/>
</div>


### 1. 减少非矩阵乘法操作（non-matmul FLOPs）
优化细节：Flash Attention 2 通过调整算法，减少了非矩阵乘法操作的浮点运算次数。例如，在前向传播中，Flash Attention 1 需要在每个块的每次迭代中执行 rescale 操作，这涉及到除法运算。而 Flash Attention 2 将 rescale 操作延后到循环的最后才执行一次，从而减少了每次迭代中的除法运算。  
原理：GPU 有专用的矩阵乘法计算单元（如 Tensor Core），其吞吐量是非矩阵乘法吞吐量的 16 倍。因此，减少非矩阵乘法操作并尽可能多地执行矩阵乘法操作可以显著提高效率。

### 2. 更好的并行化策略
#### warp介绍
线程（Thread）：在 GPU 中，线程是最小的执行单元，每个线程可以独立执行计算任务。  
Warp：多个线程（通常是 32 个线程）被组织成一个 warp。这些线程在同一个时钟周期内执行相同的指令，但可以处理不同的数据。这种执行方式被称为单指令多数据（SIMD） 或单指令多线程（SIMT） 模式  
同步执行：在 GPU 中，一个 warp 内的所有线程会同步执行相同的指令。这意味着，如果一个线程在执行某条指令时需要等待（例如，等待内存访问完成），整个 warp 都会暂停执行，直到该线程完成等待。  
分支发散（Branch Divergence）：如果一个 warp 中的线程执行了条件分支（例如，if-else 语句），并且不同线程选择了不同的分支路径，那么 GPU 会将这些线程分成多个子 warp，分别执行不同的分支路径。这种现象称为分支发散，它会导致执行效率降低


优化细节：Flash Attention 2 在序列长度维度上同时并行化前向传播和反向传播，除了批次和头数维度。这在输入序列很长（此时 batch size 通常很小）的情况下增加了 GPU 的利用率。  
在一个 attention 计算块内，Flash Attention 2 将工作分配在不同 warp 上，以减少通信和共享内存的读写。  
通过调整循环顺序，将 Q 作为外循环，K 和 V 作为内循环，每个线程块负责计算输出矩阵 O 的一部分。这种设计允许每个线程块独立进行计算，减少了线程块之间的依赖和通信需求。  
原理：通过优化工作分区，Flash Attention 2 解决了 Flash Attention 1 中不同线程块和 warp 之间工作分区不理想的问题，从而提高了 GPU 的占用率

### 3. 优化内存访问
优化细节：Flash Attention 2 通过减少共享内存的读写次数来优化内存访问。例如，它将 Q 切分为多个 warp，共享 K 和 V，避免了 warp 之间的通信。  
原理：减少共享内存的读写可以降低内存访问延迟，提高整体计算效率

### 4、优化反向传播
优化细节：Flash Attention 2 在反向传播中对 softmax 中的行方向 logsumexp 进行了微小的调整，而不是同时使用行方向的最大值和指数和。
原理：这种调整减少了反向传播中的计算量，提高了效率。


## flash-attention3改进：

Flash Attention 3 通过异步执行、交错块操作、低精度支持和硬件优化等技术，显著提升了注意力机制的计算效率和精度。它在 H100 GPU 上实现了 1.5-2.0 倍的加速，并在低精度下保持了高数值稳定性。这些改进使得 Flash Attention 3 成为 Transformer 模型中注意力机制的一个重要进步。